In [2]:
import mysql.connector
from datetime import datetime
import mysql.connector
import pandas as pd
import warnings
from tqdm.notebook import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import time
import numpy as np 
import matplotlib.gridspec as gridspec
from matplotlib.widgets import Button
import matplotlib.dates as mdates
import gymnasium as gym
import math
import random
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from gym import spaces

plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

In [3]:

class TradingEnv(gym.Env):
    def __init__(self,table_num =  0,  window_size=200):
        super(TradingEnv, self).__init__()
        
        
        # today = "A" + datetime.now().strftime("%Y%m%d") 
        self.host = '127.0.0.1'
        self.user = 'root'
        self.databaseA = 'a20240112'
        self.password = '93150lbm!!'
        # sample_code = '452190'
        
        self.window_size = window_size
        self.current_step = window_size
        
        self.action_space = spaces.Discrete(2) 

        self.tables = [code for (code,) in self.get_table_names(self.host,self.user,self.password,self.databaseA)]
        
        self.sample_code = self.tables[table_num]
        
        
        self.df = self.table_to_dataframe(self.host,self.user,self.password,self.databaseA,self.sample_code)
        self.process_df = self.preprocessing(self.df)
        print(self.process_df)
        
        
        
        

        
    def step(self,action):
        self.reward = 0 
        self.done = False
        current_price = 'Fluctuation_15s'
        
        
        fluctuation = self.process_df['Fluctuation_15s'][self.current_step]
        
        if action == 0 : # buy 
            if fluctuation > 0 : 
                self.reward = fluctuation
            elif fluctuation <= 0 : 
                self.reward = - fluctuation
        elif action == 1 : # sell
            if fluctuation < 0 : 
                self.reward = - fluctuation
            elif fluctuation > 0 : 
                self.reward = fluctuation
                
        
        
        
        if self.current_step >= len(self.process_df)-1:
            self.done = True 
        
        else : 
            self.current_step += 1
        
        return self._next_observation(), self.reward, self.done, {}
        
        
    def reset(self):
        self.current_step = self.window_size
        self.current_step += 1
        return self._next_observation()
        
    def _next_observation(self):
        # end = min(len(self.process_df), self.current_step)  # Ensure end does not exceed DataFrame length
        # start = max(0, end - self.window_size)
        start = self.current_step - self.window_size
        end = self.current_step
        obs = self.process_df.loc[start:end-1,'signal'].values
        obs = np.array(obs)
        # print(f"Start: {start}, End: {end}, Obs length: {len(obs)}")
        return obs
        
    
    
    
    def preprocessing(self,df):
        df['체결시간'] = pd.to_datetime(df['체결시간'])
        df['현재가'] = pd.to_numeric(df['현재가'].str.replace('+','').str.replace('-',''), errors='coerce')
        df['거래량'] = pd.to_numeric(df['거래량'], errors='coerce')
        df['거래대금'] = df['현재가'] * df['거래량']
        df = df.sort_values(by='체결시간', ignore_index=True)
        df['time_diff_seconds'] = df['체결시간'].diff().dt.total_seconds()
        df['exp_minus'] = np.exp(-df['time_diff_seconds'])
        df['signal'] = df['exp_minus'] * df['거래대금']

        
        df_use = df[['체결시간','거래대금', 'time_diff_seconds', '현재가','signal']]
        
        
        df_use['Fluctuation_15s'] = None

        
        intervals = 15  # 15 seconds, 30 seconds, 1 minute
        seconds = intervals 
        column_name = 'Fluctuation_15s'
        for index, row in tqdm(df_use.iterrows(),total= len(df_use)):
            base_time = row['체결시간']
            comparison_index = self.find_closest_index(df_use, base_time, seconds)
            fluctuation_rate = self.calculate_fluctuation_rate(df_use, index, comparison_index)
            df_use.at[index, column_name] = fluctuation_rate
        
        df_use = df_use.dropna()
        
        filtered_df = df_use[df_use['체결시간'].dt.time < pd.to_datetime('15:19').time()]

        filtered_df = filtered_df.dropna()
        # df_use['Fluctuation_15s'] = None
        # df_use['Fluctuation_30s'] = None
        # df_use['Fluctuation_60s'] = None
        
        # intervals = [15, 30, 60]  # 15 seconds, 30 seconds, 1 minute
        # for index, row in tqdm(df_use.iterrows(),total= len(df_use)):
        #     base_time = row['체결시간']
        #     for seconds, column_name in zip(intervals, ['Fluctuation_15s', 'Fluctuation_30s', 'Fluctuation_60s']):
        #         comparison_index = self.find_closest_index(df_use, base_time, seconds)
        #         fluctuation_rate = self.calculate_fluctuation_rate(df_use, index, comparison_index)
        #         df_use.at[index, column_name] = fluctuation_rate
        
        # df_use = df_use.dropna()
        
        
        return filtered_df 
        
        
        
        
    
    def find_closest_index(self,df, base_time, delta_seconds):
        target_time = base_time + pd.Timedelta(seconds=delta_seconds)
        closest_index = df['체결시간'].searchsorted(target_time, side='left')
        if closest_index < len(df):
            return closest_index
        return None  # Indicates no suitable index was found

    # Calculate fluctuation rate
    def calculate_fluctuation_rate(self,df, base_index, comparison_index):
        if comparison_index is None or comparison_index >= len(df):
            return np.nan  # No data available for comparison
        initial_price = df.iloc[base_index]['현재가']
        new_price = df.iloc[comparison_index]['현재가']
        return ((new_price - initial_price) / initial_price) * 100
    
    
    
    
    
    def table_to_dataframe(self,host, user, password, database, table_name):
        conn = None
        try:
            # Establish a database connection
            conn = mysql.connector.connect(
                host=host,
                user=user,
                password=password,
                database=database
            )

            # Use backticks around the table name in the query
            query = f"SELECT * FROM `{table_name}`"

            # Use pandas to read the query into a DataFrame
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                df = pd.read_sql(query, conn)

            return df

        except mysql.connector.Error as error:
            print(f"Error: {error}")
            return None
        finally:
            if conn.is_connected():
                conn.close()
        

    def get_table_names(self, host, user, password, database):
        conn = None
        cursor = None
        tables = []  # Initialize tables as an empty list

        try:
            # Establish a database connection
            conn = mysql.connector.connect(
                host=host,
                user=user,
                password=password,
                database=database
            )

            # Create a cursor object
            cursor = conn.cursor()

            # Execute a query to retrieve table names
            query = f"SELECT table_name FROM INFORMATION_SCHEMA.TABLES WHERE table_schema = '{database}'"
            cursor.execute(query)

            # Fetch all the rows
            tables = cursor.fetchall()

        except mysql.connector.Error as error:
            print(f"Error: {error}")

        finally:
            # Ensure to close resources properly
            if cursor is not None:
                cursor.close()
            if conn is not None and conn.is_connected():
                conn.close()

        return tables


In [8]:
env = TradingEnv(table_num=2)

C:\Users\pari0\AppData\Local\Temp\ipykernel_25684\1268794650.py:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_use['Fluctuation_15s'] = None


  0%|          | 0/2738 [00:00<?, ?it/s]

                           체결시간     거래대금  time_diff_seconds   현재가  \
1    2024-01-12 09:41:55.207204 -1856160           4.808627  1440   
2    2024-01-12 09:41:55.207204 -8398004           0.000000  1439   
3    2024-01-12 09:41:55.208206   -41702           0.001002  1438   
4    2024-01-12 09:41:55.208206 -2838075           0.000000  1437   
5    2024-01-12 09:41:57.293832 -1000632           2.085626  1446   
...                         ...      ...                ...   ...   
2693 2024-01-12 15:18:53.396575   447900           4.499842  1493   
2694 2024-01-12 15:18:53.791248   -74600           0.394673  1492   
2695 2024-01-12 15:18:55.329681     2986           1.538433  1493   
2696 2024-01-12 15:18:58.803315     1493           3.473634  1493   
2697 2024-01-12 15:18:58.928480    14930           0.125165  1493   

            signal Fluctuation_15s  
1    -1.514451e+04       -0.138889  
2    -8.398004e+06       -0.069493  
3    -4.166024e+04             0.0  
4    -2.838075e+06     

In [9]:
# matplotlib 설정
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

# GPU를 사용할 경우
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [10]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))


class ReplayMemory(object):

    def __init__(self, capacity):
        self.memory = deque([], maxlen=capacity)

    def push(self, *args):
        """transition 저장"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [11]:
class DQN(nn.Module):

    def __init__(self, n_observations, n_actions):
        super(DQN, self).__init__()
        self.layer1 = nn.Linear(n_observations, 128*2)
        self.layer2 = nn.Linear(128*2, 128*4)
        self.layer3 = nn.Linear(128*4, 128*2)
        self.layer4 = nn.Linear(128*2, 128)
        self.layer5 = nn.Linear(128, 64)
        self.layer6 = nn.Linear(64, n_actions)

    # 최적화 중에 다음 행동을 결정하기 위해서 하나의 요소 또는 배치를 이용해 호촐됩니다.
    # ([[left0exp,right0exp]...]) 를 반환합니다.
    def forward(self, x):
        x = F.relu(self.layer1(x)) 
        x = F.relu(self.layer2(x)) 
        x = F.relu(self.layer3(x)) 
        x = F.relu(self.layer4(x)) 
        x = F.relu(self.layer5(x)) 
        return self.layer6(x)

In [12]:
# BATCH_SIZE는 리플레이 버퍼에서 샘플링된 트랜지션의 수입니다.
# GAMMA는 이전 섹션에서 언급한 할인 계수입니다.
# EPS_START는 엡실론의 시작 값입니다.
# EPS_END는 엡실론의 최종 값입니다.
# EPS_DECAY는 엡실론의 지수 감쇠(exponential decay) 속도 제어하며, 높을수록 감쇠 속도가 느립니다.
# TAU는 목표 네트워크의 업데이트 속도입니다.
# LR은 ``AdamW`` 옵티마이저의 학습율(learning rate)입니다.
BATCH_SIZE = 1024
GAMMA = 0.99
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 1000
TAU = 0.005
LR = 1e-4

# gym 행동 공간에서 행동의 숫자를 얻습니다.
n_actions = env.action_space.n
# 상태 관측 횟수를 얻습니다.
state = env.reset()
n_observations = len(state)


# print(state)
# print(type(state))
# print(state.shape)



policy_net = DQN(n_observations, n_actions).to(device)
target_net = DQN(n_observations, n_actions).to(device)

policy_net.load_state_dict(torch.load(f'policy_net.pth'))
target_net.load_state_dict(policy_net.state_dict())

optimizer = optim.AdamW(policy_net.parameters(), lr=LR, amsgrad=True)
memory = ReplayMemory(260000)


steps_done = 0


def select_action(state):
    global steps_done
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * \
        math.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1
    if sample > eps_threshold:
        with torch.no_grad():
            # t.max (1)은 각 행의 가장 큰 열 값을 반환합니다.
            # 최대 결과의 두번째 열은 최대 요소의 주소값이므로,
            # 기대 보상이 더 큰 행동을 선택할 수 있습니다.
            
            return policy_net(state).max(1)[1].view(1, 1)
    else:
        return torch.tensor([[env.action_space.sample()]], device=device, dtype=torch.long)


episode_durations = []


def plot_durations(show_result=False):
    plt.figure(1)
    durations_t = torch.tensor(episode_durations, dtype=torch.float)
    if show_result:
        plt.title('Result')
    else:
        plt.clf()
        plt.title('Training...')
    plt.xlabel('Episode')
    plt.ylabel('Duration')
    plt.plot(durations_t.numpy())
    # 100개의 에피소드 평균을 가져 와서 도표 그리기
    if len(durations_t) >= 100:
        means = durations_t.unfold(0, 100, 1).mean(1).view(-1)
        means = torch.cat((torch.zeros(99), means))
        plt.plot(means.numpy())

    plt.pause(0.001)  # 도표가 업데이트되도록 잠시 멈춤
    if is_ipython:
        if not show_result:
            display.display(plt.gcf())
            display.clear_output(wait=True)
        else:
            display.display(plt.gcf())

In [13]:
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
    # detailed explanation). 이것은 batch-array의 Transitions을 Transition의 batch-arrays로
    # 전환합니다.
    batch = Transition(*zip(*transitions))

    # 최종이 아닌 상태의 마스크를 계산하고 배치 요소를 연결합니다
    # (최종 상태는 시뮬레이션이 종료 된 이후의 상태)
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device=device, dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None])
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    # Q(s_t, a) 계산 - 모델이 Q(s_t)를 계산하고, 취한 행동의 열을 선택합니다.
    # 이들은 policy_net에 따라 각 배치 상태에 대해 선택된 행동입니다.
    state_action_values = policy_net(state_batch).gather(1, action_batch)

    # 모든 다음 상태를 위한 V(s_{t+1}) 계산
    # non_final_next_states의 행동들에 대한 기대값은 "이전" target_net을 기반으로 계산됩니다.
    # max(1)[0]으로 최고의 보상을 선택하십시오.
    # 이것은 마스크를 기반으로 병합되어 기대 상태 값을 갖거나 상태가 최종인 경우 0을 갖습니다.
    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    with torch.no_grad():
        next_state_values[non_final_mask] = target_net(non_final_next_states).max(1)[0]
    # 기대 Q 값 계산
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Huber 손실 계산
    criterion = nn.SmoothL1Loss()
    loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))

    # 모델 최적화
    optimizer.zero_grad()
    loss.backward()
    # 변화도 클리핑 바꿔치기
    torch.nn.utils.clip_grad_value_(policy_net.parameters(), 100)
    optimizer.step()

In [205]:
true_count = 0 
total_cnt = 0 
done = False
N = 1000
if torch.cuda.is_available():
    num_episodes = 600
else:
    num_episodes = 50

for i_episode in range(num_episodes):
    # 환경과 상태 초기화
    state = env.reset()
    
    state = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
    
    
    while not  done  : 
    
        action = select_action(state)
        observation, reward, terminated, _ = env.step(action.item())
        reward = torch.tensor([reward], device=device)
        done = terminated
        
        if terminated:
            next_state = None
        else:
            next_state = torch.tensor(observation, dtype=torch.float32, device=device).unsqueeze(0)

        # 메모리에 변이 저장
        memory.push(state, action, next_state, reward)
        
        if reward > 0 : 
            true_count += 1
        
        total_cnt += 1      
        

        # 다음 상태로 이동
        state = next_state

        # (정책 네트워크에서) 최적화 한단계 수행
        optimize_model()

        # 목표 네트워크의 가중치를 소프트 업데이트
        # θ′ ← τ θ + (1 −τ )θ′
        target_net_state_dict = target_net.state_dict()
        policy_net_state_dict = policy_net.state_dict()
        for key in policy_net_state_dict:
            target_net_state_dict[key] = policy_net_state_dict[key]*TAU + target_net_state_dict[key]*(1-TAU)
        target_net.load_state_dict(target_net_state_dict)
        
        
        if total_cnt % N == 0:  # Replace N with the frequency of prints (e.g., 100)
            print(f"Step :  {total_cnt}, mid_acc : {true_count/total_cnt*100:.2f}%")
        
        if done : 
            print(f'acc : {true_count/total_cnt*100:.2f}%')
            
     
    torch.save(policy_net.state_dict(), f'policy_net.pth')
    break
            
            



Step :  1000, mid_acc : 74.00%
Step :  2000, mid_acc : 69.30%
acc : 67.99%


In [206]:
model = DQN(n_observations,n_actions)
model.load_state_dict(torch.load(f'policy_net.pth'))
model.eval() 

DQN(
  (layer1): Linear(in_features=200, out_features=256, bias=True)
  (layer2): Linear(in_features=256, out_features=512, bias=True)
  (layer3): Linear(in_features=512, out_features=256, bias=True)
  (layer4): Linear(in_features=256, out_features=128, bias=True)
  (layer5): Linear(in_features=128, out_features=64, bias=True)
  (layer6): Linear(in_features=64, out_features=2, bias=True)
)

In [207]:
total_cnt= 0 
true_cnt = 0 

for i in tqdm(range(len(env.process_df))):

    try: 
        signal_data = env.process_df['signal'][i:i+200].values
        fluc = env.process_df['Fluctuation_15s'][i:i+200].values[-1]


        input_data = torch.tensor(signal_data,dtype=torch.float32)
        action = model(input_data)
        # print(fluc)
        # print(action.argmax())
        if fluc > 0 and action.argmax() > 0 :
            true_cnt += 1
            
        elif fluc < 0 and action.argmax() < 0 : 
            total_cnt +=1
        
    except : 
        break
    
print(f'{true_cnt/total_cnt*100:.2f}%')

  0%|          | 0/2697 [00:00<?, ?it/s]

18.90%


In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

for i in range(30,40):



    env = TradingEnv(table_num=i)
    
    BATCH_SIZE = 1024
    GAMMA = 0.99
    EPS_START = 0.9
    EPS_END = 0.05
    EPS_DECAY = 1000
    TAU = 0.005
    LR = 1e-4

    n_actions = env.action_space.n
    state = env.reset()
    n_observations = len(state)



    policy_net = DQN(n_observations, n_actions).to(device)
    target_net = DQN(n_observations, n_actions).to(device)

    policy_net.load_state_dict(torch.load(f'policy_net.pth'))
    target_net.load_state_dict(policy_net.state_dict())

    optimizer = optim.AdamW(policy_net.parameters(), lr=LR, amsgrad=True)
    memory = ReplayMemory(260000)


    steps_done = 0
    
    true_count = 0 
    total_cnt = 0 
    done = False
    N = 1000

    state = env.reset()

    state = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)


    while not  done  : 

        action = select_action(state)
        observation, reward, terminated, _ = env.step(action.item())
        reward = torch.tensor([reward], device=device)
        done = terminated
        
        if terminated:
            next_state = None
        else:
            next_state = torch.tensor(observation, dtype=torch.float32, device=device).unsqueeze(0)

        # 메모리에 변이 저장
        memory.push(state, action, next_state, reward)
        
        if reward > 0 : 
            true_count += 1
        
        total_cnt += 1      
        

        # 다음 상태로 이동
        state = next_state

        # (정책 네트워크에서) 최적화 한단계 수행
        optimize_model()

        # 목표 네트워크의 가중치를 소프트 업데이트
        # θ′ ← τ θ + (1 −τ )θ′
        target_net_state_dict = target_net.state_dict()
        policy_net_state_dict = policy_net.state_dict()
        for key in policy_net_state_dict:
            target_net_state_dict[key] = policy_net_state_dict[key]*TAU + target_net_state_dict[key]*(1-TAU)
        target_net.load_state_dict(target_net_state_dict)
        
        
        if total_cnt % N == 0:  # Replace N with the frequency of prints (e.g., 100)
            print(f"Step :  {total_cnt}, mid_acc : {true_count/total_cnt*100:.2f}%")
        
        if done : 
            print(f'acc : {true_count/total_cnt*100:.2f}%')
    
    
    total_cnt= 0 
    true_cnt = 0 

    for i in tqdm(range(len(env.process_df))):
        total_cnt += 1 
        try: 
            signal_data = env.process_df['signal'][i:i+200].values
            fluc = env.process_df['Fluctuation_15s'][i:i+200].values[-1]


            input_data = torch.tensor(signal_data,dtype=torch.float32)
            action = model(input_data)
            # print(fluc)
            # print(action.argmax())
            if fluc > 0 and action.argmax() == 0 :
                true_cnt += 1
                
            elif fluc < 0 and action.argmax() == 1 : 
                true_cnt +=1
            
        except : 
            break
        
        
    print(f'EVAL : {true_cnt/total_cnt*100:.2f}%')     
    torch.save(policy_net.state_dict(), f'policy_net.pth')

        
            


C:\Users\pari0\AppData\Local\Temp\ipykernel_25684\1268794650.py:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_use['Fluctuation_15s'] = None


  0%|          | 0/7240 [00:00<?, ?it/s]

                           체결시간     거래대금  time_diff_seconds   현재가  \
1    2024-01-12 09:41:33.675269    47000           0.523127  4700   
2    2024-01-12 09:41:34.103196    94000           0.427927  4700   
3    2024-01-12 09:41:34.655392  1099800           0.552196  4700   
4    2024-01-12 09:41:35.133178    18800           0.477786  4700   
5    2024-01-12 09:41:35.278073    47000           0.144895  4700   
...                         ...      ...                ...   ...   
7188 2024-01-12 15:18:52.093593  9190000          11.997245  4595   
7189 2024-01-12 15:18:54.330773  5004800           2.237180  4600   
7190 2024-01-12 15:18:58.101781  4995600           3.771008  4600   
7191 2024-01-12 15:18:59.392030    -4590           1.290249  4590   
7192 2024-01-12 15:18:59.602439   -64260           0.210409  4590   

             signal Fluctuation_15s  
1      27855.226134        0.106383  
2      61274.745884        0.106383  
3     633137.503754        0.106383  
4      11658.912043

  0%|          | 0/7192 [00:00<?, ?it/s]

EVAL : 0.00%


C:\Users\pari0\AppData\Local\Temp\ipykernel_25684\1268794650.py:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_use['Fluctuation_15s'] = None


  0%|          | 0/27364 [00:00<?, ?it/s]

                            체결시간      거래대금  time_diff_seconds    현재가  \
1     2024-01-12 09:41:34.070282   -241600           0.992402  60400   
2     2024-01-12 09:41:35.085073  -5375600           1.014791  60400   
3     2024-01-12 09:41:35.219073    302500           0.134000  60500   
4     2024-01-12 09:41:35.436073  -1328800           0.217000  60400   
5     2024-01-12 09:41:35.701061 -13771200           0.264988  60400   
...                          ...       ...                ...    ...   
27203 2024-01-12 15:18:56.091254  29836800           0.090630  59200   
27204 2024-01-12 15:18:56.092252   2960000           0.000998  59200   
27205 2024-01-12 15:18:56.092252    947200           0.000000  59200   
27206 2024-01-12 15:18:56.092252    296000           0.000000  59200   
27207 2024-01-12 15:18:56.092252  10123200           0.000000  59200   

             signal Fluctuation_15s  
1     -8.955755e+04        0.165563  
2     -1.948538e+06        0.165563  
3      2.645635e+05  

KeyboardInterrupt: 